### Analyzing all-dates descriptives

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import datetime as dt
import time

from importlib import reload
from Utils import TransantiagoConstants
from RunSilentlyDailyEtapasBuilder import RunSilentlyDailyEtapasBuilderClass

In [3]:
DTPMDir = TransantiagoConstants.DTPMDir
dates = TransantiagoConstants.updateCurrentSSHDates()

codes_path = os.path.join(DTPMDir, 'codes_services.xlsx')
codes = pd.read_excel(codes_path, encoding = 'latin-1')

In [4]:
codes.loc[:,'TS_CODE'] = codes.loc[:,'TS_CODE'].apply(str)

* Not necessary again

In [5]:
#tic = time.clock()

#for date in dates:
#    etapas_builder = RunSilentlyDailyEtapasBuilderClass(date)
#    etapas_builder.runSimplifyEtapas()
#    del etapas_builder

#toc = time.clock()
#toc-tic

* Continuing with analyses.
* <strike>Consider to refactor this block in order to get number of observations.</strike>
* <strike>Consider to refactor this to get number of trx per database.</strike>
* TODO: Consider to refactor this to get number of trx per database per operator -> <strike>this could be computationally expensive and not-so-easy, since it would be necessary to have some dictionary of operators and routes, and search between etapas and the dictionary</strike>
* <strike>Consider to refactor this to get number of already processed databases.</strike>
* Methodology is not robust to potential databases particularities. <strike><strong>Be aware of 2017-04-10 and 2017-04-17: outliers</strong></strike>. Identification of issues with these particular databases should be done from outside
* Identification of special dates should be done from outside.
* Complete process (for all available dates) took about 35 minutes.

* Filtering to refactor

In [6]:
dates = dates[:1]

* End filtering

In [34]:
summary_descriptives = pd.DataFrame(index=dates)

In [35]:
for date in dates:
    etapas_builder = RunSilentlyDailyEtapasBuilderClass(date)
    [etapas_df, processed_sorted_df, filtered_df, filtered_turnstile_df, filtered_no_turnstile_df]=etapas_builder.runCompleteProcess()
    
    #Pre-processing processed_sorted_df. Be aware that etapas_df only contains obs. of type 'BUS'
    processed_sorted_df.loc[:,'servicio_subida'] = processed_sorted_df.loc[:,'servicio_subida'].str.replace('T','')
#    processed_sorted_df.loc[:,'servicio_subida'] = processed_sorted_df.loc[:,'servicio_subida'].str.replace('00','')
    processed_sorted_df.loc[:,'TS_CODE'] = processed_sorted_df.loc[:,'servicio_subida'].str.split(' ').str[0]
#    processed_sorted_df.loc[:,'DIRECTION'] = processed_sorted_df.loc[:,'servicio_subida'].str.split(' ').str[1]
    processed_sorted_df.loc[:,'DIRECTION'] = processed_sorted_df.loc[:,'servicio_subida'].str[-1:]
    processed_sorted_df.loc[:,'DIRECTION'] = processed_sorted_df.loc[:,'DIRECTION'].str.replace('R','Ret')
    processed_sorted_df.loc[:,'DIRECTION'] = processed_sorted_df.loc[:,'DIRECTION'].str.replace('I','Ida')
    merged_etapas_df = pd.merge(processed_sorted_df,codes, on=['TS_CODE','DIRECTION'], how='left')
    merged_etapas_df.loc[(merged_etapas_df['UN'].isnull()) & (merged_etapas_df['TS_CODE'].str[:1]=='1'),'UN'] = 1
    merged_etapas_df.loc[(merged_etapas_df['UN'].isnull()) & (merged_etapas_df['TS_CODE'].str[:1]=='2'),'UN'] = 2
    merged_etapas_df.loc[(merged_etapas_df['UN'].isnull()) & (merged_etapas_df['TS_CODE'].str[:1]=='3'),'UN'] = 3
    merged_etapas_df.loc[(merged_etapas_df['UN'].isnull()) & (merged_etapas_df['TS_CODE'].str[:1]=='4'),'UN'] = 4
    merged_etapas_df.loc[(merged_etapas_df['UN'].isnull()) & (merged_etapas_df['TS_CODE'].str[:1]=='5'),'UN'] = 5
    merged_etapas_df.loc[(merged_etapas_df['UN'].isnull()) & (merged_etapas_df['TS_CODE'].str[:1]=='B'),'UN'] = 6
    merged_etapas_df.loc[(merged_etapas_df['UN'].isnull()) & (merged_etapas_df['TS_CODE'].str[:1]=='F'),'UN'] = 7    
    
    summary_descriptives.loc[date,'total_raw_trx'] = len(etapas_df.index)
    summary_descriptives.loc[date,'total_clean_trx'] = len(processed_sorted_df.index)
    summary_descriptives.loc[date,'total_turns_trx'] = len(processed_sorted_df[processed_sorted_df['si_2017_torniquete']==True].index)
    summary_descriptives.loc[date,'total_no_turns_trx'] = len(processed_sorted_df[processed_sorted_df['si_2017_torniquete']==False].index)    
    

    trx_by_un = merged_etapas_df.groupby(['UN'])['index'].count()
    summary_descriptives['trx_1'], summary_descriptives['trx_2'], summary_descriptives['trx_3'] = [trx_by_un.loc[1], trx_by_un.loc[2], trx_by_un.loc[3]]
    summary_descriptives['trx_4'], summary_descriptives['trx_5'], summary_descriptives['trx_6'] = [trx_by_un.loc[4], trx_by_un.loc[5], trx_by_un.loc[6]]
    summary_descriptives['trx_7'] = trx_by_un.loc[7]
    

Not found in turnstile database: 1614656


In [36]:
processed_sorted_df.head()

,index,id,nviaje,tipo_transporte,t_subida,servicio_subida,par_subida,sitio_subida,fecha_instalacion,mismo_paradero,misma_patente,mismo_servicio,diferencia_tiempo,diferencia_tiempo_secs,si_torniquete,si_2017_torniquete,TS_CODE,DIRECTION
0,1863684,3552888222,1,BUS,2017-03-01 17:39:47,F07 00R,L-34-89-2-PO,BBJZ-74,NaT,False,False,False,NaT,NaN,False,False,F07,Ret
1,1203472,2529602798,1,BUS,2017-03-01 17:44:11,F07 00R,L-34-53-25-PO,BBJZ-74,NaT,False,True,True,NaT,NaN,False,False,F07,Ret
2,1598210,3169360958,1,BUS,2017-03-01 17:44:13,F07 00R,L-34-53-25-PO,BBJZ-74,NaT,True,True,True,00:00:02,2.0,False,False,F07,Ret
3,930514,2140206880,2,BUS,2017-03-01 17:44:16,F07 00R,L-34-53-25-PO,BBJZ-74,NaT,True,True,True,00:00:03,3.0,False,False,F07,Ret
4,1571797,3149482742,1,BUS,2017-03-01 17:46:33,F07 00R,L-34-53-40-PO,BBJZ-74,NaT,False,True,True,NaT,NaN,False,False,F07,Ret


In [37]:
merged_etapas_df.head()

,index,id,nviaje,tipo_transporte,t_subida,servicio_subida,par_subida,sitio_subida,fecha_instalacion,mismo_paradero,misma_patente,mismo_servicio,diferencia_tiempo,diferencia_tiempo_secs,si_torniquete,si_2017_torniquete,TS_CODE,DIRECTION,UN,USER_CODE
0,1863684,3552888222,1,BUS,2017-03-01 17:39:47,F07 00R,L-34-89-2-PO,BBJZ-74,NaT,False,False,False,NaT,NaN,False,False,F07,Ret,7.0,F07
1,1203472,2529602798,1,BUS,2017-03-01 17:44:11,F07 00R,L-34-53-25-PO,BBJZ-74,NaT,False,True,True,NaT,NaN,False,False,F07,Ret,7.0,F07
2,1598210,3169360958,1,BUS,2017-03-01 17:44:13,F07 00R,L-34-53-25-PO,BBJZ-74,NaT,True,True,True,00:00:02,2.0,False,False,F07,Ret,7.0,F07
3,930514,2140206880,2,BUS,2017-03-01 17:44:16,F07 00R,L-34-53-25-PO,BBJZ-74,NaT,True,True,True,00:00:03,3.0,False,False,F07,Ret,7.0,F07
4,1571797,3149482742,1,BUS,2017-03-01 17:46:33,F07 00R,L-34-53-40-PO,BBJZ-74,NaT,False,True,True,NaT,NaN,False,False,F07,Ret,7.0,F07


In [38]:
len(processed_sorted_df.index)

2210399

In [39]:
len(merged_etapas_df.index)

2210399

In [40]:
merged_etapas_df.loc[merged_etapas_df['UN'].isnull(),'TS_CODE'].unique()

array([], dtype=object)

In [41]:
len(merged_etapas_df[merged_etapas_df['UN'].isnull()].index)

0

In [42]:
summary_descriptives['trx_1']+summary_descriptives['trx_2']+summary_descriptives['trx_3']+summary_descriptives['trx_4']+summary_descriptives['trx_5']+summary_descriptives['trx_6']+summary_descriptives['trx_7']

2017-03-01    2210399
dtype: int64

In [43]:
summary_descriptives

,total_raw_trx,total_clean_trx,total_turns_trx,total_no_turns_trx,trx_1,trx_2,trx_3,trx_4,trx_5,trx_6,trx_7
2017-03-01,2329229.0,2210399.0,48762.0,2161637.0,253643,362604,395000,424460,397512,243354,133826


In [ ]:
quantile = 0.97
processed = 0

tic = time.clock()

for date in dates:
    etapas_builder = RunSilentlyDailyEtapasBuilderClass(date)
    [etapas_df, processed_sorted_df, filtered_df, filtered_turnstile_df, filtered_no_turnstile_df]=etapas_builder.runCompleteProcess()
    
    #Pre-processing processed_sorted_df. Be aware that etapas_df only contains obs. of type 'BUS'
    processed_sorted_df.loc[:,'servicio_subida'] = processed_sorted_df.loc[:,'servicio_subida'].str.replace('T','')
    processed_sorted_df.loc[:,'servicio_subida'] = processed_sorted_df.loc[:,'servicio_subida'].str.replace('00','')
    processed_sorted_df.loc[:,'TS_CODE'] = processed_sorted_df.loc[:,'servicio_subida'].str.split(' ').str[0]
    processed_sorted_df.loc[:,'DIRECTION'] = processed_sorted_df.loc[:,'servicio_subida'].str.split(' ').str[1]
    processed_sorted_df.loc[:,'DIRECTION'] = processed_sorted_df.loc[:,'DIRECTION'].str.replace('R','Ret')
    processed_sorted_df.loc[:,'DIRECTION'] = processed_sorted_df.loc[:,'DIRECTION'].str.replace('I','Ida')
    merged_etapas_df = pd.merge(processed_sorted_df,codes, on=['TS_CODE','DIRECTION'], how='left')

    #Filling trx info:
    total_raw_trx.append(len(etapas_df.index))
    total_trx.append(len(processed_sorted_df.index))
    total_turns_trx.append(processed_sorted_df[processed_sorted_df.si_2017_torniquete==True].count()[0])
    total_no_turns_trx.append(processed_sorted_df[processed_sorted_df.si_torniquete==False].count()[0])
        
    #Filtering and following the original methodology:
    turnstile_q = filtered_turnstile_df.diferencia_tiempo.quantile(quantile)
    no_turnstile_q = filtered_no_turnstile_df.diferencia_tiempo.quantile(quantile)    
    
    filtered_turnstile_q_df = filtered_turnstile_df[filtered_turnstile_df.diferencia_tiempo<=turnstile_q]
    filtered_no_turnstile_q_df = filtered_no_turnstile_df[filtered_no_turnstile_df.diferencia_tiempo<=no_turnstile_q]
    
    turnstile_obs.append(len(filtered_turnstile_q_df.index))
    turnstile_means.append(filtered_turnstile_q_df['diferencia_tiempo_secs'].mean())
    turnstile_stds.append(filtered_turnstile_q_df['diferencia_tiempo_secs'].std())
    turnstile_medians.append(filtered_turnstile_q_df['diferencia_tiempo_secs'].quantile(0.50))
    turnstile_maxs.append(filtered_turnstile_q_df['diferencia_tiempo_secs'].max())
    
    no_turnstile_obs.append(len(filtered_no_turnstile_q_df.index))
    no_turnstile_means.append(filtered_no_turnstile_q_df['diferencia_tiempo_secs'].mean())
    no_turnstile_stds.append(filtered_no_turnstile_q_df['diferencia_tiempo_secs'].std())    
    no_turnstile_medians.append(filtered_no_turnstile_q_df['diferencia_tiempo_secs'].quantile(0.50))
    no_turnstile_maxs.append(filtered_no_turnstile_q_df['diferencia_tiempo_secs'].max())
    
    processed = processed + 1
    print('Number of already processed ddbb is ' + str(processed))
    del etapas_builder
    
toc = time.clock()  
toc-tic

* Summarizing

In [ ]:
dates = [dt.datetime.strptime(x, "%Y-%m-%d").date() for x in dates]

In [ ]:
summary_descriptives = pd.DataFrame(
    {'date': dates,
     'total_raw_trx': total_raw_trx,
     'total_trx': total_trx,
     'total_turns_trx': total_turns_trx,
     'total_no_turns_trx': total_no_turns_trx,
     'turnstile_obs':turnstile_obs,
     'turnstile_means': turnstile_means,
     'turnstile_stds': turnstile_stds,
     'turnstile_medians': turnstile_medians,
     'turnstile_maxs': turnstile_maxs,
     'no_turnstile_obs': no_turnstile_obs,
     'no_turnstile_means': no_turnstile_means,
     'no_turnstile_stds': no_turnstile_stds,
     'no_turnstile_medians': no_turnstile_medians,
     'no_turnstile_maxs': no_turnstile_maxs})

In [ ]:
cols = ['date','total_raw_trx','total_trx','total_turns_trx','total_no_turns_trx',
        'turnstile_obs','turnstile_means','turnstile_stds','turnstile_medians','turnstile_maxs',
        'no_turnstile_obs','no_turnstile_means','no_turnstile_stds', 'no_turnstile_medians', 'no_turnstile_maxs']
summary_descriptives = summary_descriptives[cols]

* Checking

In [ ]:
summary_descriptives

* Writting the ddbb to a file

In [ ]:
#summary_descriptives.to_csv('C:/Users/Tesista/Desktop/Evasion/01_analisis/03_datos/05_SUMMARY/DescriptivesSummary.csv', sep=';', encoding = 'latin-1')